@Copyright IQIYI 2021
http://challenge.ai.iqiyi.com/

In [6]:
import pandas as pd
import numpy as np
import json
import math
import tensorflow as tf
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

In [7]:
data_dir = "./data/new_data_B/"
model_dir = "./data/model/best_model.h5"
submit_dir = "./data/submit/"

In [8]:
#制作一个迭代器，迭代器里面的每个元素是一个bt=n的step
#https://blog.csdn.net/weixin_37737254/article/details/103884255
class DataGenerator(Sequence):
    def __init__(self, df, batch_size):
        self.data = df
        self.num = df.shape[0]
        self.batch_size = batch_size
        self.fea = ['father_id_score', 'cast_id_score', 'tag_score',
       'device_type', 'device_ram', 'device_rom', 'sex', 'age', 'education',
       'occupation_status', 'territory_score','launch_times', 
       'launch_times_31', 'launch_times_15', 'launch_times_7', 'playtime_31',
       'playtime_15', 'playtime_7']#'launch_date_len_target_enc','start_end_launch',目前最佳只有钱18个,'launch_date_len','launch_type_0', 'launch_type_1'

    def __len__(self):
        return math.ceil(self.num / self.batch_size)

    def __getitem__(self,idx):
        batch_data = self.data.iloc[idx*self.batch_size:(idx+1)*self.batch_size]

        input_1 = np.array([i for i in batch_data.launch_seq_31])
        input_2 = np.array([i for i in batch_data.playtime_seq])
        input_3 = np.array([i for i in batch_data.duration_prefer])
        input_4 = np.array([i for i in batch_data.interact_prefer])
        input_5 = np.array(batch_data[self.fea])
        #以上特征要做成[[][][]]这样的形式读取
        
        output = np.array(batch_data.label)

        return (input_1, input_2, input_3, input_4, input_5), output

In [19]:
# def build_model(seq_feature_num, seq_len, feature_num):
#     input_1 = tf.keras.Input(shape=(seq_len, seq_feature_num))
#     output_1 = tf.keras.layers.GRU(64)(input_1)

#     input_2 = tf.keras.Input(shape=(feature_num, ))
#     layer = tf.keras.layers.Dense(256, activation="elu")(input_2)
#     layer = tf.keras.layers.Dense(128, activation="elu")(layer)
#     output_2 = tf.keras.layers.Dense(64, activation="elu")(layer)

#     output = tf.concat([output_1, output_2], -1)
#     output = tf.keras.layers.Dense(1, activation="relu")(output)

#     model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

#     return model

In [20]:
# def my_model(seq_feature_num, seq_len, feature_num):
#     input_1 = tf.keras.Input(shape=(seq_len, seq_feature_num))
#     output_1 = tf.keras.layers.GRU(64,return_sequences=True)(input_1)
#     bn1 = tf.keras.layers.BatchNormalization()(output_1)
#     dp1 = tf.keras.layers.Dropout(0.15)(bn1)
    
#     output_11 = tf.keras.layers.GRU(64)(dp1)
#     bn11 = tf.keras.layers.BatchNormalization()(output_11)
#     dp11 = tf.keras.layers.Dropout(0.15)(bn11)

#     input_2 = tf.keras.Input(shape=(feature_num, ))
#     layer = tf.keras.layers.Dense(256, activation="elu")(input_2)
#     layer = tf.keras.layers.Dense(128, activation="elu")(layer)
#     output_2 = tf.keras.layers.Dense(64, activation="elu")(layer)
#     bn2 = tf.keras.layers.BatchNormalization()(output_2)
#     dp2 = tf.keras.layers.Dropout(0.2)(bn2)
    
#     output = tf.concat([dp11, dp2], -1)
#     output = tf.keras.layers.Dense(1, activation="relu")(output)

#     model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

#     return model

In [9]:
def build_model(seq_len,dur_seq_len,inter_seq_len, feature_num):
    input_1 = tf.keras.Input(shape=(seq_len,1))
    output_1 = tf.keras.layers.GRU(32)(input_1)

    input_2 = tf.keras.Input(shape=(seq_len,1))
    output_2 = tf.keras.layers.GRU(32)(input_2)
    
    input_3 = tf.keras.Input(shape=(inter_seq_len,1))
    output_3 = tf.keras.layers.GRU(11)(input_3)  #11
    
    input_4 = tf.keras.Input(shape=(dur_seq_len,1))
    output_4 = tf.keras.layers.GRU(16)(input_4)  #16
    
    input_5 = tf.keras.Input(shape=(feature_num, ))
    output_5 = tf.keras.layers.Dense(64, activation="relu")(input_5)

    output = tf.concat([output_1, output_2,output_3,output_4,output_5], -1)
#     output = tf.keras.layers.Dense(128, activation="relu")(output)
#     dp = tf.keras.layers.Dropout(0.15)(output)去掉涨了0.002
    output = tf.keras.layers.Dense(64, activation="relu")(output)
    output = tf.keras.layers.Dense(1, activation="relu")(output)

    model = tf.keras.Model(inputs=[input_1, input_2,input_3, input_4,input_5], outputs=output)

    return model

# train

In [22]:
# train data
train = pd.read_csv(data_dir + "train_data.txt", sep="\t")
train["launch_seq_31"] = train.launch_seq_31.apply(lambda x: json.loads(x))
train["playtime_seq"] = train.playtime_seq.apply(lambda x: json.loads(x))
train["duration_prefer"] = train.duration_prefer.apply(lambda x: json.loads(x))
train["interact_prefer"] = train.interact_prefer.apply(lambda x: json.loads(x))

In [23]:
train.head()

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10000000,211,2,16,"[131, 132, 141, 164, 179, 185, 187, 189, 191, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.849842,0.874528,-0.611652,-0.527202,...,0.0,0.194677,1.038058,-0.142070,-0.956632,0.827448,-0.545979,0.744456,-1.798070,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000001,173,0,3,"[114, 117, 118]","[0, 0, 0]",-0.410684,-0.372292,-0.611652,-0.527202,...,0.0,2.427026,-2.051382,-1.254643,1.045333,-0.319627,2.052733,-1.343260,1.567918,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000002,128,1,1,[128],[0],-0.604611,-0.564111,-0.611652,-0.527202,...,0.0,-2.037672,-1.046499,1.515868,1.045333,0.827448,2.052733,0.744456,1.080016,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000003,199,0,2,"[144, 144]","[1, 0]",-0.507648,-0.564111,1.117465,1.164124,...,0.0,0.194677,-0.719528,-0.813574,-0.956632,0.827448,-0.545979,0.744456,-1.344670,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10000004,167,0,4,"[154, 155, 156, 157]","[0, 0, 0, 0]",-0.313720,-0.276383,-0.611652,-0.527202,...,0.0,0.194677,2.876669,3.917349,1.045333,-0.319627,-0.545979,-1.343260,-0.376322,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [24]:
# shuffle data
train = train.sample(frac=1).reset_index(drop=True)
#data

In [25]:
test = pd.read_csv(data_dir + "test_data.txt", sep="\t")
test["launch_seq_31"] = test.launch_seq_31.apply(lambda x: json.loads(x))
test["playtime_seq"] = test.playtime_seq.apply(lambda x: json.loads(x))
test["duration_prefer"] = test.duration_prefer.apply(lambda x: json.loads(x))
test["interact_prefer"] = test.interact_prefer.apply(lambda x: json.loads(x))

In [26]:
train.launch_times_15

0        -0.798152
1         0.331995
2         0.331995
3         2.874825
4         0.049458
            ...   
599996    1.462141
599997    1.462141
599998   -0.233079
599999   -0.515615
600000   -0.233079
Name: launch_times_15, Length: 600001, dtype: float64

In [27]:
train.launch_seq_15

0         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0]
2         [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0]
4         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0]
                                ...                       
599996    [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
599997    [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0]
599998    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
599999    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
600000    [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: launch_seq_15, Length: 600001, dtype: object

In [13]:
#由于之前的内存问题，再次做一部分特征
#计算launch_date的序列长度

# launch_date_len = []
# for i in train.launch_date:
#     launch_date_len.append(len(i))
# train['launch_date_len'] = launch_date_len

# launch_date_len = []
# for i in test.launch_date:
#     launch_date_len.append(len(i))
# test['launch_date_len'] = launch_date_len

# def kfold_mean(df_train, df_test, target, target_mean_list):
#     folds = StratifiedKFold(n_splits=5)

#     mean_of_target = df_train[target].mean()

#     for fold_, (trn_idx, val_idx) in tqdm(enumerate(folds.split(df_train, y=df_train['label']))):
#         tr_x = df_train.iloc[trn_idx, :]
#         vl_x = df_train.iloc[val_idx, :]

#         for col in target_mean_list:
#             df_train.loc[vl_x.index, f'{col}_target_enc'] = vl_x[col].map(tr_x.groupby(col)[target].mean())

#     for col in target_mean_list:
#         df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)

#         df_test[f'{col}_target_enc'] = df_test[col].map(df_train.groupby(col)[f'{col}_target_enc'].mean())

#         df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)
#     return df_train, df_test

# feature_list =  ['launch_date_len']
# train,test = kfold_mean(data.iloc[:600001],data.iloc[600001:],'label',feature_list)

#分开归一化launch_date_len，因为训练集和测试集差的较多
# maxtrainlaunch = max(train.launch_date_len)
# mintrainlaunch = min(train.launch_date_len)
# train['launch_date_len'] = train['launch_date_len'].apply(lambda x:(x-mintrainlaunch)/(maxtrainlaunch-mintrainlaunch))

# maxtestlaunch = max(test.launch_date_len)
# mintestlaunch = min(test.launch_date_len)
# test['launch_date_len'] = test['launch_date_len'].apply(lambda x:(x-mintestlaunch)/(maxtestlaunch-mintestlaunch))

# train.head()

In [28]:
test.head()

,user_id,end_date,label,launch_date_len,launch_date,launch_type,launch_times,launch_type_0,launch_type_1,launch_type_01rate,...,tag_score,device_type,device_ram,device_rom,sex,age,education,occupation_status,territory_score,interact_prefer
0,10354569,187,-1,8,"[121, 122, 136, 138, 141, 143, 157, 167]","[0, 0, 0, 0, 0, 0, 0, 0]",0.074134,0.107254,-0.611652,-0.527202,...,0.0,0.194677,1.105658,-0.130098,1.045333,-0.319627,2.052733,-1.34326,-0.643756,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10594476,175,-1,7,"[108, 111, 157, 157, 168, 169, 170]","[0, 0, 1, 0, 0, 0, 0]",-0.022830,-0.084564,1.117465,-0.043966,...,0.0,0.194677,-0.814455,-0.840461,-0.956632,-0.319627,-0.545979,-1.34326,-0.507040,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10032866,177,-1,2,"[103, 122]","[0, 0]",-0.507648,-0.468201,-0.611652,-0.527202,...,0.0,-2.037672,-1.123687,1.515394,1.045333,-1.466701,-0.545979,-1.34326,2.133736,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10467002,185,-1,6,"[122, 129, 136, 141, 142, 143]","[0, 0, 0, 0, 0, 0]",-0.119793,-0.084564,-0.611652,-0.527202,...,0.0,-2.037672,-1.541749,-1.421985,1.045333,0.827448,-0.545979,-1.34326,-0.458544,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,10520906,208,-1,8,"[134, 159, 162, 165, 181, 187, 192, 202]","[0, 0, 0, 0, 0, 0, 0, 0]",0.074134,0.107254,-0.611652,-0.527202,...,0.0,0.194677,1.010731,1.262390,1.045333,0.827448,-0.545979,-1.34326,1.036742,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [15]:
# user_lbe = LabelEncoder()
# data['user_id'] = user_lbe.fit_transform(data['user_id'])
# data['user_id'] = data['user_id'] + 1

In [16]:
# generator_test = DataGenerator(data[:20], batch_size=8)
# for i, item in enumerate(iter(generator_test)):
#     if(i == len(generator_test)):
#         break
#     (input_1, input_2,input_3, input_4,input_5), output = item
#     print(i, input_1.shape, input_2.shape, input_3.shape, input_4.shape, input_5.shape)
#     print(i, output.shape, output)

In [29]:
def aiyiqi_metric(y_true,y_pred):
    y_true = list(y_true)
    y_pred = list(y_pred)
    score = 0
    for i in range(len(y_true)):
        score += abs(y_true[i]-y_pred[i])/7
    return 100*(1-score/len(y_true))

In [30]:
new_test = DataGenerator(test,100)

new_train = DataGenerator(train[:594000],100)
new_val = DataGenerator(train.iloc[594000:],100)
        
model = build_model(seq_len=32,dur_seq_len=16,inter_seq_len=11,feature_num=18)
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(0.0008),loss="mse",metrics=["mse"])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_mse", patience=3, restore_best_weights=True)
lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(patience=2,monitor='val_mse', factor=0.1)
best_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_dir,save_best_only=True, save_weights_only=False,verbose=1)
#model.fit(iter(train_bt),steps_per_epoch=len(train_bt),validation_data=iter(val_bt),validation_steps=len(val_bt),epochs=20,callbacks=[best_checkpoint,early_stopping,lr_reduce])
#model.save('./data/model/model_fold{}.h5'.format(kf))
model.fit_generator(generator=new_train,
                    steps_per_epoch=len(new_train),
                    epochs=20,
                    verbose=1,
                    validation_data=new_val,
                    validation_steps=len(new_val),
#                     use_multiprocessing=False,
#                     workers=1,
                    callbacks=[best_checkpoint,early_stopping,lr_reduce])
    
#重新加载当前折最优的模型
best_model = tf.keras.models.load_model(model_dir)
#测试集推理
test_pred =  best_model.predict(new_test, steps=len(new_test))[:,0]
 
#验证集推理
val_pred =  best_model.predict(new_val, steps=len(new_val))[:,0]

#计算整体验证集得分
y_true = train.iloc[594000:]['label']
score = aiyiqi_metric(y_true,val_pred)
print('得分：{}'.format(score))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 1)]      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 32, 1)]      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 11, 1)]      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 16, 1)]      0                                            
______________________________________________________________________________________________

D:\Anaconda3\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


5940/5940 [==============================] - 57s 9ms/step - loss: 1.6688 - mse: 1.6688 - val_loss: 1.7188 - val_mse: 1.7188

Epoch 00001: val_loss improved from inf to 1.71877, saving model to ./data/model\best_model.h5
Epoch 2/20
5940/5940 [==============================] - 55s 9ms/step - loss: 1.6278 - mse: 1.6278 - val_loss: 1.7094 - val_mse: 1.7094

Epoch 00002: val_loss improved from 1.71877 to 1.70941, saving model to ./data/model\best_model.h5
Epoch 3/20
5940/5940 [==============================] - 55s 9ms/step - loss: 1.6188 - mse: 1.6188 - val_loss: 1.7152 - val_mse: 1.7152

Epoch 00003: val_loss did not improve from 1.70941
Epoch 4/20
5940/5940 [==============================] - 59s 10ms/step - loss: 1.6115 - mse: 1.6115 - val_loss: 1.7000 - val_mse: 1.7000

Epoch 00004: val_loss improved from 1.70941 to 1.69998, saving model to ./data/model\best_model.h5
Epoch 5/20
5940/5940 [==============================] - 61s 10ms/step - loss: 1.6065 - mse: 1.6065 - val_loss: 1.7124 - va

# predict

In [5]:
new_test_B = DataGenerator(test,100)
#重新加载当前折最优的模型
best_model = tf.keras.models.load_model(model_dir)
#测试集推理
test_pred =  best_model.predict(new_test_B, steps=len(new_test_B))[:,0]

In [31]:
#保存测试集结果
prediction = test_pred
test['prediction'] = list(prediction)
res = test[["user_id", "prediction"]]
res.to_csv(submit_dir + "submit_B.csv", index=False, header=False, float_format="%.2f")